In [1]:
#Importing relevant packages
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate as tb
from queue import PriorityQueue

In [148]:
class PuzzleNode: 
    
    def __init__(self, current_state, g_eval, h_eval, parent_node=None, in_explored_set=False):
        '''
        Initialize a puzzle node instance
        
        Args: 
            current_state (list of lists): the current state of the puzzle node
            g_eval (int): the evaluation of the current cost
            h_eval (int): the evaluation of the heuristic function, expressing the expected cost
            parent_node (an object): initially None, poiting to the object that is the parent node
            in_explored_set (bool): whether the current node has been expanded or not
            
        Outputs: a puzzle node object
                    
        '''
        self.current_state = current_state
        self.parent_node = parent_node
        self.all_cost = g_eval + h_eval
        self.current_cost = g_eval
        self.predicted_cost = h_eval
        self._in_explored_set = in_explored_set
        
    def __lt__(self, other):
        '''
        Comparing the total cost of a node instance with another node instance 
        
        Args: another node instance
        
        Outputs: a boolean indicates whether the current node total cost is smaller compared to the other node
        '''
        return self.all_cost < other.all_cost

    def __str__(self):
        '''
        Converting the state of the current node to string
        
        Args: None
        
        Outputs: a string version of the current node state
        '''
        return str(self.current_state)

    def visualize(self):
        '''
        Printing out the current node state
        
        Args: None
        
        Outputs: None
        '''
        print(tb(self.current_state, tablefmt="grid"))
       
    
def validity_test(state, size):
    '''
    Testing the validity of the input state
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        size (int): the dimension of the puzzle table (size*size)
        
    Outputs: True if the state passed the validity test, False otherwise
    '''
    
    state = np.array(state)
    #Testing if the state has an appropriate number of row/column, and of appropriate datatype
    if state.dtype == 'object': 
        return True
    else: 
        result = np.zeros(shape=(len(state), len(state)), dtype=int)
        i = 0
        for row in state: 
            result[i, :] = row
            i += 1
        #Testing if the state has exactly size*size number of tile
        bool_node_shape = (result.shape == (size, size))
        #Testing if the state tile values create a valid puzzle table
        bool_node_state = (np.sort(result.flatten()) == np.array(range(size**2))).all()
        if bool_node_shape & bool_node_state: 
            return True
        else: 
            return False
        
def solvability_test(state): 
    '''
    Testing the solvability of the input state
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        
    Outputs: True if the state is solvable, False otherwise
    '''
    #Flatten the state, disregarding the 0
    new_state = list(np.array(state).flatten())
    new_state.pop(new_state.index(0))
    
    #Counting the number of inversion by looping over all tile values
    inversion = 0
    for i in range(len(new_state)): 
        for j in range(i, len(new_state)): 
            if new_state[i] > new_state[j]: 
                inversion += 1
                
    #If the number of inversion is odd then the puzzle is solvable, otherwise not
    if (inversion % 2 == 0):
        return True
    else: 
        return False

def hfunc_Manhattan(state):
    '''
    Calculating the Manhattan distance of a state of the puzzle
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        
    Outputs: a numerical value of the heuristic function, signifying the 
             combined Manhanttan distances of all tiles
    '''
    distance = 0
    #Looping throuhg all the tiles to get the Manhattan distance
    for row in range(len(state)): 
        for col in range(len(state)): 
            distance += np.sum(np.abs(np.array([row, col]) - \
                                      np.array([state[row][col]//len(state), state[row][col]%len(state)])))
    return distance

def hfunc_misplaced(state):
    '''
    Calculating the number of misplaced tiles of a state of the puzzle
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        
    Outputs: a numerical value of the heuristic function, signifying the number of misplaced tiles
    '''
    state_array = np.array(state).flatten()
    misplace = np.equal(state_array, np.arange(len(state)**2))
    return len(state)**2 - sum(misplace)
    
#Creating a handler for the heuristic function    
heuristics = [hfunc_Manhattan, hfunc_misplaced]
    
def generate_next_state(state):
    '''
    Generating valid next states according to the rule of the puzzle game
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        
    Outputs: a list of valid next state, according to the rule of the game
    '''
    state_array = np.array(state)
    row, col = np.where(state_array == 0)[0][0], np.where(state_array == 0)[1][0]
    next_states = []
    #If 0 not in the first row, then 0 can be swaped with a tile above it
    if row != 0: 
        next_state = [item.copy() for item in state]
        next_state[row][col], next_state[row-1][col] = next_state[row-1][col], next_state[row][col]
        next_states.append(next_state)
    #If 0 not in the last row, then 0 can be swaped with a tile below it
    if (row != (len(state)-1)): 
        next_state = [item.copy() for item in state]
        next_state[row][col], next_state[row+1][col] = state[row+1][col], state[row][col]
        next_states.append(next_state)
    #If 0 not in the first column, then 0 can be swaped with a tile to the left of it
    if col != 0:
        next_state = [item.copy() for item in state]
        next_state[row][col], next_state[row][col-1] = state[row][col-1], state[row][col]
        next_states.append(next_state)
    #If 0 not in the last column, then 0 can be swaped with a tile to the right of it
    if (col != (len(state)-1)): 
        next_state = [item.copy() for item in state] 
        next_state[row][col], next_state[row][col+1] = state[row][col+1], state[row][col]
        next_states.append(next_state)
    return next_states
    
def generate_goal_state(n): 
    '''
    Generating the goal state of a puzzle game
        
    Args: 
        n (int): the dimension of the puzzle table (size*size)
        
    Outputs: the goal state of the table, where every tile is in order
    '''
    array = np.arange(n*n).reshape(n,n)
    goal_state = [[] for _ in range(n)]
    for _ in range(len(goal_state)): 
        goal_state[_] = list(array[_,:])
    return goal_state

def solvePuzzle(n, state, heuristic, prnt=False): 
    '''
    A* algorithm to solve the n-puzzle
        
    Args: 
        n (int): the dimension of the puzzle table (size*size)
        state (list of lists): an arbitrary state of the puzzle
        heuristic (func): a heuristic function used to speed up A* search
        prnt (bool): whether we should print the optimal solution or not
        
    Outputs: 
        the number of moves we need to make to reach the optimal solution, if we do it optimally
        the worst case frontier size
        an error code
    '''
    #Testing the validity of the input state
    if not validity_test(state, n): 
        return None, None, -1
    
    #Testing the solvability of the input state 
    if not solvability_test(state): 
        return None, None, -2
    
    #Initializing the initial node
    initial_node = PuzzleNode(state, 0, heuristic(state))
    #Creating a priority queue to keep track of the order to which we should expand the nodes
    frontier = PriorityQueue()
    frontier.put(initial_node)
    #Counting the number of expansion made
    num_expansion = 0
    #Creating a cost database to make sure we just expand the optimal path
    recorded_cost = {str(initial_node.current_state):initial_node}
    frontier_size = frontier.qsize()
    #Creating the goal state 
    goal_state = generate_goal_state(n)
    
    #Main loop, while the frontier is not empty, then expand
    while not frontier.empty():
        current_node = frontier.get()
        #If the current node is already explore, then expand other nodes in the frontier
        if current_node._in_explored_set: 
            continue
        #If the current node is the goal state, then exit the loop
        if current_node.current_state == goal_state: 
            break
        
        #Generating valid next states
        next_states = generate_next_state(current_node.current_state)
        #Looping over the next states
        for state in next_states:
            #Since we expand once, then the cost is increase by 1
            cost = current_node.current_cost + 1
            
            #If the node is in the cost database then check if the current cost is less than the recorded cost
            #so we can not expand it in the next while loop call
            if str(state) in recorded_cost: 
                if recorded_cost[str(state)].current_cost > cost: 
                    recorded_cost[str(state)]._in_explored_set = True
                else: 
                    continue
            #Creating a child node for the next state, setting the parent to the current node
            child_node = PuzzleNode(state, cost, heuristic(state), current_node)
            #Putting the next node to the frontier, then record its cost to the database
            frontier.put(child_node)
            recorded_cost[str(child_node.current_state)] = child_node
        num_expansion += 1
        #Getting the maximum frontier size
        frontier_size = max(frontier.qsize(), frontier_size)
        
    #Reconstructing the optimal solution, by tracing the parent node of the goal node
    optimal_solution = [current_node]
    while current_node.parent_node: 
        optimal_solution.append(current_node.parent_node)
        current_node = current_node.parent_node
    
    #Printing out diagnostic information
    print("Solved after {} expansions.\n".format(num_expansion))
    print("Worst frontier size is {}.\n".format(frontier_size))
    print("Require {} step(s) to reach the goal state optimally.\n".format(len(optimal_solution)))
    
    #Printing out the steps to solution
    if prnt == True: 
        optimal_solution = optimal_solution[::-1]
        i = 1
        for node in optimal_solution: 
            print('Step {}:'.format(i))
            node.visualize()
            i += 1

    return len(optimal_solution), frontier_size, 0

In [149]:
steps, frontier_size, err = solvePuzzle(3, [[5,7,6],[2,4,3],[8,1,0]], heuristics[0], prnt=True)

Solved after 4193 expansions.

Worst frontier size is 2390.

Require 29 step(s) to reach the goal state optimally.

Step 1:
+---+---+---+
| 5 | 7 | 6 |
+---+---+---+
| 2 | 4 | 3 |
+---+---+---+
| 8 | 1 | 0 |
+---+---+---+
Step 2:
+---+---+---+
| 5 | 7 | 6 |
+---+---+---+
| 2 | 4 | 0 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 3:
+---+---+---+
| 5 | 7 | 0 |
+---+---+---+
| 2 | 4 | 6 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 4:
+---+---+---+
| 5 | 0 | 7 |
+---+---+---+
| 2 | 4 | 6 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 5:
+---+---+---+
| 5 | 4 | 7 |
+---+---+---+
| 2 | 0 | 6 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 6:
+---+---+---+
| 5 | 4 | 7 |
+---+---+---+
| 2 | 6 | 0 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 7:
+---+---+---+
| 5 | 4 | 0 |
+---+---+---+
| 2 | 6 | 7 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 8:
+---+---+---+
| 5 | 0 | 4 |
+---+---+---+
| 2 | 6 | 7 |
+---+---+---+
| 8 | 1 | 3 |
+---+---+---+
Step 9:
+---+---+---+
| 0 | 5 | 4 |


In [150]:
steps, frontier_size, err = solvePuzzle(3, [[7,0,8],[4,6,1],[5,3,2]], heuristics[0], prnt=True)

Solved after 3118 expansions.

Worst frontier size is 1804.

Require 26 step(s) to reach the goal state optimally.

Step 1:
+---+---+---+
| 7 | 0 | 8 |
+---+---+---+
| 4 | 6 | 1 |
+---+---+---+
| 5 | 3 | 2 |
+---+---+---+
Step 2:
+---+---+---+
| 7 | 8 | 0 |
+---+---+---+
| 4 | 6 | 1 |
+---+---+---+
| 5 | 3 | 2 |
+---+---+---+
Step 3:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 4 | 6 | 0 |
+---+---+---+
| 5 | 3 | 2 |
+---+---+---+
Step 4:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 4 | 0 | 6 |
+---+---+---+
| 5 | 3 | 2 |
+---+---+---+
Step 5:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 4 | 3 | 6 |
+---+---+---+
| 5 | 0 | 2 |
+---+---+---+
Step 6:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 4 | 3 | 6 |
+---+---+---+
| 0 | 5 | 2 |
+---+---+---+
Step 7:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 0 | 3 | 6 |
+---+---+---+
| 4 | 5 | 2 |
+---+---+---+
Step 8:
+---+---+---+
| 7 | 8 | 1 |
+---+---+---+
| 3 | 0 | 6 |
+---+---+---+
| 4 | 5 | 2 |
+---+---+---+
Step 9:
+---+---+---+
| 7 | 8 | 1 |


In [153]:
steps, frontier_size, err = solvePuzzle(3, [[2,3,7],[1,8,0],[6,5,4]], heuristics[0], prnt=True)

Solved after 182 expansions.

Worst frontier size is 109.

Require 18 step(s) to reach the goal state optimally.

Step 1:
+---+---+---+
| 2 | 3 | 7 |
+---+---+---+
| 1 | 8 | 0 |
+---+---+---+
| 6 | 5 | 4 |
+---+---+---+
Step 2:
+---+---+---+
| 2 | 3 | 7 |
+---+---+---+
| 1 | 8 | 4 |
+---+---+---+
| 6 | 5 | 0 |
+---+---+---+
Step 3:
+---+---+---+
| 2 | 3 | 7 |
+---+---+---+
| 1 | 8 | 4 |
+---+---+---+
| 6 | 0 | 5 |
+---+---+---+
Step 4:
+---+---+---+
| 2 | 3 | 7 |
+---+---+---+
| 1 | 0 | 4 |
+---+---+---+
| 6 | 8 | 5 |
+---+---+---+
Step 5:
+---+---+---+
| 2 | 0 | 7 |
+---+---+---+
| 1 | 3 | 4 |
+---+---+---+
| 6 | 8 | 5 |
+---+---+---+
Step 6:
+---+---+---+
| 0 | 2 | 7 |
+---+---+---+
| 1 | 3 | 4 |
+---+---+---+
| 6 | 8 | 5 |
+---+---+---+
Step 7:
+---+---+---+
| 1 | 2 | 7 |
+---+---+---+
| 0 | 3 | 4 |
+---+---+---+
| 6 | 8 | 5 |
+---+---+---+
Step 8:
+---+---+---+
| 1 | 2 | 7 |
+---+---+---+
| 3 | 0 | 4 |
+---+---+---+
| 6 | 8 | 5 |
+---+---+---+
Step 9:
+---+---+---+
| 1 | 2 | 7 |
+-

In [154]:
steps, frontier_size, err = solvePuzzle(3, [[5,7,6],[2,4,3],[8,1,0]], heuristics[1])

Solved after 61479 expansions.

Worst frontier size is 20782.

Require 29 step(s) to reach the goal state optimally.



In [155]:
steps, frontier_size, err = solvePuzzle(3, [[7,0,8],[4,6,1],[5,3,2]], heuristics[1])

Solved after 27859 expansions.

Worst frontier size is 12330.

Require 26 step(s) to reach the goal state optimally.



In [182]:
steps, frontier_size, err = solvePuzzle(3, [[2,3,7],[1,8,0],[6,5,4]], heuristics[1])

Solved after 689 expansions.

Worst frontier size is 420.

Require 18 step(s) to reach the goal state optimally.



# Comparing Performance Of Heuristic Functions

The frontier of the examples for the Manhattan distance heuristic is: 2390, 1804, 109.

The frontier of the examples for the misplaced tile heuristic is: 20702, 12330, 420. 

The number of expansion of the examples for the Manhattan distance heuristic is: 4193, 3118, 182. 

The number of expansion of the examples for the misplaced tile heuristic is: 61479, 27859, 689.

Both heuristics agree on the same number of steps for optimal solution.

It is quite clear that the Manhattan distance is a better heuristic for this problem, which makes sense since the Manhattan distance is always greater than or equal to the number of misplace tiles, making it a tighter bound for the actual cost and therefore better. This is true because for every misplaced tile (misplace heuristic evaluate to +1), the Manhattan distance is always greater than or equal to 1 (Manhattan distance + more than 1), and since both heuristics are admissible the heuristic with greater values is more beneficial because it informs the algorithm an expected cost nearer to the actual cost.

# Solvability Of The Puzzle

We can prove that some configuration of the puzzle is unsolvable by proving that the state space of the puzzle can be separated into two disjoint sets, each of which cannot evolve into the other by any admissible rules. Defining the inversion value of a specific configuration to be the number of pair in the state for which the first value of the pair is greater than the second value, going from left to right (meaning that the tiles in the pair is essentially misplaced) - excluding the 0 tile. We can see that the inversion value can be either odd or even. We will now show that the inversion number is invariant under transformation via an admissible move.

If the move is moving from either to the left or the right by one tile, then the only tiles moved is the 0 tile and another tile, which will not change the respective position of the tiles when flatten out, therefore does not change the inversion value. If the move is moving either up or down, then the respective positions of the tiles in both the rows above the move position and the rows below the move position changes, creating a net increase or decrease of an even number for the inversion value. So the inversion value stays the same. Since these two exhaust the valid move list, we just proved that the parity value is invariant under any valid transformation of the state. Thus, the state with different inversion value will belong to disjoint set in the state space and each state in each set is unreachable by any state in the other set. Thus, the goal state, with the inversion number of 0 (even), can only be reached via an intial state of even inversion. This check is implemented in the solvability_test function. 

In [201]:
#Pattern database
import itertools

def construct_all_possible_subproblem():
    '''
    Constructing the possible subproblem of the relaxed problem for containing tile 1,2,3,4 (and the empty tile)
    
    Args: None 
    
    Outputs: a list of all possible subproblem
    
    '''
    subproblems = {}
    all_indices = itertools.permutations(range(9), 5) 
    for indices in all_indices:
        state = [[np.inf,np.inf,np.inf], [np.inf,np.inf,np.inf], [np.inf,np.inf,np.inf]]
        state[indices[0]//3][indices[0]%3] = 0
        state[indices[1]//3][indices[1]%3] = 1
        state[indices[2]//3][indices[2]%3] = 2
        state[indices[3]//3][indices[3]%3] = 3
        state[indices[4]//3][indices[4]%3] = 4
        subproblems[indices] = state
    return subproblems
    
def solve_subproblem(state, heuristic): 
    '''
    Solving an instance of the relaxed problem, essentially just adapting the solvePuzzle function
    
    Args: 
        state (list of lists): an arbitrary state of the relaxed problem
        heuristic (func): the heuristic function
        
    Outputs: the actual cost to solve the relaxed problem
    
    '''
    if not solvability_test(state): 
        return 0
    goal_state = [[0,1,2],[3,4,np.inf],[np.inf, np.inf, np.inf]]
    initial_node = PuzzleNode(state, 0, heuristic(state))
    frontier = PriorityQueue()
    frontier.put(initial_node)
    num_expansion = 0
    recorded_cost = {str(initial_node.current_state):initial_node}
    frontier_size = frontier.qsize()
    
    while not frontier.empty():
        current_node = frontier.get()
        if current_node._in_explored_set: 
            continue
        if current_node.current_state == goal_state: 
            break
        
        next_states = generate_next_state(current_node.current_state)
        for state in next_states: 
            cost = current_node.current_cost + 1
            if str(state) in recorded_cost: 
                if recorded_cost[str(state)].current_cost > cost: 
                    recorded_cost[str(state)]._in_explored_set = True
                else: 
                    continue
            child_node = PuzzleNode(state, cost, heuristic(state), current_node)
            frontier.put(child_node)
            recorded_cost[str(child_node.current_state)] = child_node
        num_expansion += 1
        frontier_size = max(frontier.qsize(), frontier_size)
    
    optimal_solution = [current_node]
    while current_node.parent_node: 
        optimal_solution.append(current_node.parent_node)
        current_node = current_node.parent_node

    return len(optimal_solution)

def construct_pattern_database(): 
    '''
    Creating a pattern database
    
    Args: None
    
    Outputs: a dictionary with keys as the position indices of the relaxed problem tiles, 
            and the values as the costs
    
    '''
    subproblems = construct_all_possible_subproblem()
    database = {}
    for subproblem in subproblems: 
        cost = solve_subproblem(subproblems[subproblem], heuristics[1])
        database[subproblem] = cost
    return database

pattern_database = construct_pattern_database()

In [203]:
def hfunc_pdatabase(state): 
    '''
    Calculating heuristic cost by comparing with the pattern database
        
    Args: 
        state (list of lists): an arbitrary state of the puzzle
        
    Outputs: a numerical value of the heuristic function, signifying actual cost of the relaxed problem
    '''
    state_array = list(np.array(state).flatten())
    indices = (state_array.index(0), state_array.index(1), state_array.index(2),\
               state_array.index(3), state_array.index(4))
    return pattern_database[indices]

heuristics.append(hfunc_pdatabase)

Solved after 51715 expansions.

Worst frontier size is 23921.

Require 29 step(s) to reach the goal state optimally.



In [206]:
steps, frontier_size, err = solvePuzzle(3, [[5,7,6],[2,4,3],[8,1,0]], heuristics[2])

Solved after 51715 expansions.

Worst frontier size is 23921.

Require 29 step(s) to reach the goal state optimally.



In [205]:
steps, frontier_size, err = solvePuzzle(3, [[7,0,8],[4,6,1],[5,3,2]], heuristics[2])

Solved after 17307 expansions.

Worst frontier size is 9906.

Require 26 step(s) to reach the goal state optimally.



In [204]:
steps, frontier_size, err = solvePuzzle(3, [[2,3,7],[1,8,0],[6,5,4]], heuristics[2])

Solved after 2288 expansions.

Worst frontier size is 1499.

Require 18 step(s) to reach the goal state optimally.



# Pattern Database Heuristics

A more effective strategy for heuristic is to construct a pattern database by solving easier problems: relaxed instances of the original problem, then record these actual cost to solve the relaxed problems in a database to compare the original problem state to when we are solving them. We can see that the pattern database heuristics for 4 tiles above is slightly worst than the Manhattan distance, and much better than the misplace tile heuristics in terms of both the number of expansions and the size of the worst frontier. However, solving the pattern database and storing its value might become intractable for larger problems (for this toy problem, the size of the database is around 15,000 already). 